<img src="./img/metro_madrid.jpg" alt="drawing" width="500"/>

# Transporte y renta en la Comunidad de Madrid

### TEMA

Este proyecto busca arrojar luz sobre los patrones de comportamiento en el transporte de los ciudadanos de la Comunidad de Madrid, estudiando posibles vínculos con su renta. Para ello, se utiliza la [Encuesta de Movilidad del Consorcio Regional](https://datos.comunidad.madrid/dataset/resultados-edm2018) de Transportes de Madrid (CRTM), realizada en el 2018.

Para los cálculos de la renta, se utilizan diferentes tablas extraídas de la [Encuesta de Condiciones de Vida](https://ine.es/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736176807&menu=ultiDatos&idp=1254735976608) del INE, para el año 2018

### HIPÓTESIS

La hipótesis principal de este proyecto es que **existe una relación entre la renta y el uso del transporte en la Comunidad de Madrid**. En concreto, se buscará contrastar:
+ Si las rentas más altas eligen el coche como primera opción para desplazamientos laborales y por placer.
+ Las rentas bajas, para trabajar, utilizan más el transporte público que el coche.
+ Las mujeres utilizan más el transporte público para desplazamientos laborales, sin tener en cuenta la renta.
    + Lo que, unido al segundo punto, explicaría que las mujeres no usan más el transporte público por deseo sino por necesidad.
+ Los hombres utilizan más el transporte privado para desplazamientos por placer


### SUPUESTOS
Dados los datos utilizados, hemos de tener en cuenta varios puntos:
+ Se está utilizando la última encuesta publicada por el CRTM (en serio).
    + Sí se cuentan con los datos de uso del Metro de Madrid, que en 2023 volvieron a niveles pre-pandemia. Por tanto, podemos suponer que no han cambiado las pautas de comportamiento en un nivel agregado
+ No contamos con la renta de los encuestados. Para ello generaremos un dato *proxy*, que estimaremos en función de las condiciones socioeconómicas de los encuestados (edad, ocupación, género, grado de educación, y si cuenta con coche).
    + Renta != Riqueza. Estimaremos la cantidad de **generar** , no de **poseer**. Una persona sin trabajo pero con patrimonio no quedará reflejada en este estudio.
    + Por disponibilidad de los datos, se utilizarán las medias de las rentas. En función de disponibilidad de los datos, es posible que se recoja la media española en lugar de la de la Comunidad de Madrid.
+ Los datos sólo tienen en cuenta días estrictamente laborales (Lunes-Jueves)
+ Si bien no tiene impacto a primera vista sobre las hipótesis, se realizará un estudio de impacto de la **meteorología** sobre el uso de unos u otros medios transportes.


## OBTENCIÓN DE LOS DATOS

### DATASETS Y FUENTES ALTERNATIVAS DE DATOS

In [2]:
import hashlib
import requests
import datetime
import pandas as pd

In [30]:

### Importar openpyxl
# guarda en variables los datasets y su fuente
# df_transpk = pd.DataFrame(pd.read_csv("./data/raw/kaggle_public_transp.csv"))
# fuente_1a = "https://www.kaggle.com/datasets/dataguapa/madrid-public-transportation-data-2018"

### MAIN

df_transp_ind = pd.DataFrame(pd.read_excel("./data/raw/EDM2018INDIVIDUOS.xlsx", sheet_name = 'INDIVIDUOS'))
fuente_1 = "https://datos.comunidad.madrid/dataset/resultados-edm2018"

df_transp_trp = pd.DataFrame(pd.read_excel("./data/raw/EDM2018VIAJES.xlsx", sheet_name = 'VIAJES'))
fuente_1 = "https://datos.comunidad.madrid/dataset/resultados-edm2018"


### AUX

df_hogares = pd.DataFrame(pd.read_csv("./data/raw/ine_gasto_hogares.csv", sep=";"))
fuente_2 = "https://www.ine.es/jaxiT3/Tabla.htm?t=24900"

df_ingresos = pd.DataFrame(pd.read_csv("./data/raw/ine_madrid_fuentes_ingreso.csv",sep=";"))
fuente_3 = "https://ine.es/jaxiT3/Tabla.htm?t=53687"

df_renta_es = pd.DataFrame(pd.read_csv("./data/raw/ine_renta_edad_sexo.csv",encoding='latin1',sep=";"))
fuente_4 = "https://ine.es/jaxiT3/Tabla.htm?t=9942"

df_educacion = pd.DataFrame(pd.read_excel("./data/raw/ine_renta_educacion.xlsx", sheet_name="educación",skiprows=2,index_col=0,nrows=5)) ### Terminar de apañar
fuente_5 = "https://ine.es/ss/Satellite?c=INESeccion_C&cid=1259944504067&p=1254735110672&pagename=ProductosYServicios%2FPYSLayout&param1=PYSDetalleFichaIndicador&param3=1259937499084"

Muestra mediante un head() los principales datasets con los que vas a trabajar

In [12]:
#Df auxiliar 1
df_hogares.head(5)

,Gastos medios y distribución porcentual,Quintil de gasto,Códigos de gasto (2 dígitos),Periodo,Total
0,Gasto medio por hogar,Total,Índice general,2023,"32.616,66"
1,Gasto medio por hogar,Total,Índice general,2022,"31.567,71"
2,Gasto medio por hogar,Total,Índice general,2021,"29.243,61"
3,Gasto medio por hogar,Total,Índice general,2020,"26.995,76"
4,Gasto medio por hogar,Total,Índice general,2019,"30.242,76"


In [13]:
df_ingresos.head()


,Comunidades y Ciudades Autonomas,Provincias,Islas,Distribuci�n por fuente de ingresos,Periodo,Total
0,"Madrid, Comunidad de",NaN,NaN,Renta bruta media por persona,2018,"19.042,0"
1,"Madrid, Comunidad de",NaN,NaN,Fuente de ingreso: salario,2018,"12.368,0"
2,"Madrid, Comunidad de",NaN,NaN,Fuente de ingreso: pensiones,2018,"3.208,0"
3,"Madrid, Comunidad de",NaN,NaN,Fuente de ingreso: prestaciones por desempleo,2018,"228,0"
4,"Madrid, Comunidad de",NaN,NaN,Fuente de ingreso: otras prestaciones,2018,"534,0"


In [14]:
df_renta_es.head()

,Sexo,Edad,Renta anual neta media por persona y por unidad de consumo,Periodo,Total
0,Ambos sexos,Menores de 16 años,Renta neta media por persona,2018,8.919
1,Ambos sexos,De 16 a 29 años,Renta neta media por persona,2018,10.156
2,Ambos sexos,De 30 a 44 años,Renta neta media por persona,2018,11.397
3,Ambos sexos,De 45 a 64 años,Renta neta media por persona,2018,12.550
4,Ambos sexos,65 y más años,Renta neta media por persona,2018,12.758


In [16]:
df_educacion.head()

,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
Media,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Total,16587.0,17422.0,17327.0,16604.0,16428.0,16023.0,15752.0,15716.0,16129.0,16677.0,17154.0,17570.0,18538.0,18480.0,19562.0
"Nivel 0-2: preescolar, primaria y secundaria de 1ª etapa",13410.0,14071.0,13918.0,13290.0,13231.0,12926.0,12610.0,12435.0,12647.0,13052.0,13373.0,13890.0,14682.0,14357.0,15289.0
Nivel 3-4: secundaria de 2ª etapa y postsecundaria no superior,17342.0,17725.0,17627.0,16978.0,16446.0,16131.0,15713.0,15555.0,16112.0,16399.0,16638.0,16999.0,17799.0,17798.0,18617.0
Nivel 5-8: primer y segundo ciclo de educación superior y doctorado,22946.0,24515.0,24325.0,23285.0,22836.0,21838.0,21372.0,21579.0,22116.0,22783.0,23314.0,23436.0,23924.0,23890.0,25089.0


In [3]:
### AEMET Meteo.

df_weather=pd.DataFrame(pd.read_csv('./data/treated/aemet_weather.csv'))
### Change of data values so we can compare them to CRTM surveys 
df_weather['fechamerge'] = df_weather['fecha'].str.split("-")
df_weather['fechamerge']
### Expanding to three new colums
df_weather [['year','month','day']] = pd.DataFrame(df_weather.fechamerge.tolist(), index = df_weather.index)
### Drops of 0s en month y day
df_weather['month'] = df_weather['month'].str.replace("0","")
df_weather['day'] = df_weather['day'].str[0].replace("0","")+df_weather['day'].str[1]
df_weather['day'].head(50)
### Concatenate year, month, day
df_weather['fechamerge'] = df_weather['year']+"-"+df_weather['month']+"-"+df_weather['day']
### Check
df_weather.head()

,Unnamed: 0,fecha,indicativo,nombre,provincia,altitud,tmed,prec,tmin,horatmin,...,horaPresMin,hrMedia,hrMax,horaHrMax,hrMin,horaHrMin,fechamerge,year,month,day
0,0,2018-02-01,3195,"MADRID, RETIRO",MADRID,667,"6,6","0,1","2,3",07:40,...,17,69.0,81.0,Varias,47.0,16:00,2018-2-1,2018,2,1
1,1,2018-02-02,3195,"MADRID, RETIRO",MADRID,667,"4,4","0,0","1,2",23:59,...,05,45.0,60.0,Varias,35.0,13:00,2018-2-2,2018,2,2
2,2,2018-02-03,3195,"MADRID, RETIRO",MADRID,667,"5,2","2,3","0,2",02:30,...,24,51.0,76.0,23:50,40.0,13:00,2018-2-3,2018,2,3
3,3,2018-02-04,3195,"MADRID, RETIRO",MADRID,667,"3,6","17,3","1,5",13:00,...,24,90.0,97.0,Varias,76.0,00:00,2018-2-4,2018,2,4
4,4,2018-02-05,3195,"MADRID, RETIRO",MADRID,667,"1,8","14,1","0,8",11:40,...,14,88.0,96.0,Varias,75.0,06:10,2018-2-5,2018,2,5


In [ ]:
### Conection w/ AEMET

url = 'https://opendata.aemet.es/opendata/sh/b3aa9d28'
res = requests.get(url)
print(res.status_code)
# weather = res.json()
weather_meta = res.json()
weather_meta

In [ ]:
### Aemet to CSV conversor

df_weather_meta = pd.DataFrame(weather_meta)
df_weather_meta.to_csv('./data/aemet_weather_meta.csv')
df_weather_meta

In [31]:
### Individual Survey - Date conversor for merge
df_transp_ind['fechamerge'] = df_transp_ind['DANNO'].astype(str) + '-' + df_transp_ind['DMES'].astype(str) + '-' + df_transp_ind['DDIA'].astype(str)

### Concatenate columns so we get an unique id
df_transp_ind['id_indiv']=df_transp_ind['ID_HOGAR'].astype(str) + df_transp_ind['ID_IND'].astype(str)
df_transp_ind.head()

,ID_HOGAR,ID_IND,C2SEXO,EDAD_FIN,ELE_G_POND,C4NAC,C5CAM,C6CARNE,C7ESTUD,C8ACTIV,...,DMES,DANNO,DIASEM,DNOVIAJO,C11ZT1259,C12ZT1259,CPMR,TIPO_ENCUESTA,fechamerge,id_indiv
0,189,1,1,28,66.304668,1,1,4,4,1,...,4,2018,3,NaN,104-001B,NaN,2,CAPI,2018-4-25,1891
1,189,2,2,23,73.500000,1,1,4,4,4,...,4,2018,3,2.0,NaN,NaN,2,CAPI,2018-4-25,1892
2,244,1,1,36,66.304668,2,1,4,4,1,...,4,2018,4,NaN,104-001B,NaN,2,CAPI,2018-4-26,2441
3,244,2,2,35,69.473571,2,1,1,3,4,...,4,2018,4,NaN,NaN,NaN,2,CAPI,2018-4-26,2442
4,324,1,1,81,79.174900,1,1,4,2,3,...,4,2018,3,NaN,NaN,NaN,2,CAPI,2018-4-25,3241


In [37]:
### Trips Survey - Arranging ids, Column study / renaming / drop
df_transp_trp['id_indiv']=df_transp_trp['ID_HOGAR'].astype(str) + df_transp_trp['ID_IND'].astype(str)
df_transp_trp['id_trip']=df_transp_trp['id_indiv'].astype(str) + df_transp_trp['ID_VIAJE'].astype(str)
df_transp_trp.rename(columns={'VFRECUENCIA':'freq',
                              'VNOPUBLICO': 'no_public',
                              'MOTIVO_PRIORITARIO': 'reason',
                              'DISTANCIA_VIAJE': 'distance',
                              'ELE_G_POND_ESC2': 'trip_pond'},inplace=True)
df_transp_trp = df_transp_trp[['id_indiv','id_trip','freq','reason','distance','trip_pond']]
df_transp_trp

,id_indiv,id_trip,freq,reason,distance,trip_pond
0,1891,18911,1,2,6.660582,66.304668
1,1891,18912,1,2,6.660582,66.304668
2,2441,24411,1,2,6.586497,66.304668
3,2441,24412,1,2,6.586497,66.304668
4,2442,24421,1,5,0.203441,69.473571
...,...,...,...,...,...,...
222739,60111371,601113713,1,2,0.708580,46.428103
222740,60111371,601113714,1,2,3.237211,46.428103
222741,60111371,601113715,4,5,2.769001,46.428103
222742,60111371,601113716,4,2,3.893271,46.428103


In [33]:
### Merge of transp indiv and weather
df_weather_merge = df_weather[['fechamerge','tmed','prec']]
df_transp_ind = pd.merge(df_transp_ind,df_weather_merge, on = 'fechamerge', how = 'left')
df_transp_ind

,ID_HOGAR,ID_IND,C2SEXO,EDAD_FIN,ELE_G_POND,C4NAC,C5CAM,C6CARNE,C7ESTUD,C8ACTIV,...,DIASEM,DNOVIAJO,C11ZT1259,C12ZT1259,CPMR,TIPO_ENCUESTA,fechamerge,id_indiv,tmed,prec
0,189,1,1,28,66.304668,1,1,4,4,1,...,3,NaN,104-001B,NaN,2,CAPI,2018-4-25,1891,"20,8","0,0"
1,189,2,2,23,73.500000,1,1,4,4,4,...,3,2.0,NaN,NaN,2,CAPI,2018-4-25,1892,"20,8","0,0"
2,244,1,1,36,66.304668,2,1,4,4,1,...,4,NaN,104-001B,NaN,2,CAPI,2018-4-26,2441,"20,2","0,0"
3,244,2,2,35,69.473571,2,1,1,3,4,...,4,NaN,NaN,NaN,2,CAPI,2018-4-26,2442,"20,2","0,0"
4,324,1,1,81,79.174900,1,1,4,2,3,...,3,NaN,NaN,NaN,2,CAPI,2018-4-25,3241,"20,8","0,0"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85059,6011027,1,2,55,49.726989,1,1,4,7,1,...,1,NaN,079-16-431,NaN,2,CATI,2018-6-4,60110271,"16,2","0,0"
85060,6011053,1,1,49,66.566789,1,1,5,7,1,...,1,NaN,079-16-431,NaN,2,CATI,2018-6-4,60110531,"16,2","0,0"
85061,6011053,2,1,6,64.450350,1,1,1,1,6,...,1,NaN,NaN,079-07-121,2,CATI,2018-6-4,60110532,"16,2","0,0"
85062,6011059,1,1,14,64.450350,1,1,1,2,6,...,1,NaN,NaN,079-16-437,2,CATI,2018-6-4,60110591,"16,2","0,0"


In [35]:
### Indv Survey - Column study / renaming / drop
df_transp_ind.rename(columns={'C2SEXO' : 'gender',
                           'EDAD_FIN' : 'age',
                           'ELE_G_POND' : 'indiv_pond',
                           'C4NAC' : 'spanish',
                           'C7ESTUD' : 'studies',
                           'C8ACTIV' : 'activity',
                           'DDIA' : 'day',
                           'DMES' : 'month',
                           'DANNO' : 'year',
                           'DIASEM' : 'week_day',}, inplace=True)
df_transp_ind = df_transp_ind[['id_indiv','gender','age','indiv_pond','spanish','studies','activity','day','month','year','fechamerge','tmed','prec']]
df_transp_ind


,id_indiv,gender,age,indiv_pond,spanish,studies,activity,day,month,year,fechamerge,tmed,prec
0,1891,1,28,66.304668,1,4,1,25,4,2018,2018-4-25,"20,8","0,0"
1,1892,2,23,73.500000,1,4,4,25,4,2018,2018-4-25,"20,8","0,0"
2,2441,1,36,66.304668,2,4,1,26,4,2018,2018-4-26,"20,2","0,0"
3,2442,2,35,69.473571,2,3,4,26,4,2018,2018-4-26,"20,2","0,0"
4,3241,1,81,79.174900,1,2,3,25,4,2018,2018-4-25,"20,8","0,0"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
85059,60110271,2,55,49.726989,1,7,1,4,6,2018,2018-6-4,"16,2","0,0"
85060,60110531,1,49,66.566789,1,7,1,4,6,2018,2018-6-4,"16,2","0,0"
85061,60110532,1,6,64.450350,1,1,6,4,6,2018,2018-6-4,"16,2","0,0"
85062,60110591,1,14,64.450350,1,2,6,4,6,2018,2018-6-4,"16,2","0,0"


In [39]:
### Merge of indivs and trips:
df_transp = pd.merge(df_transp_trp,df_transp_ind, on="id_indiv", how="left")
df_transp

,id_indiv,id_trip,freq,reason,distance,trip_pond,gender,age,indiv_pond,spanish,studies,activity,day,month,year,fechamerge,tmed,prec
0,1891,18911,1,2,6.660582,66.304668,1,28,66.304668,1,4,1,25,4,2018,2018-4-25,"20,8","0,0"
1,1891,18912,1,2,6.660582,66.304668,1,28,66.304668,1,4,1,25,4,2018,2018-4-25,"20,8","0,0"
2,2441,24411,1,2,6.586497,66.304668,1,36,66.304668,2,4,1,26,4,2018,2018-4-26,"20,2","0,0"
3,2441,24412,1,2,6.586497,66.304668,1,36,66.304668,2,4,1,26,4,2018,2018-4-26,"20,2","0,0"
4,2442,24421,1,5,0.203441,69.473571,2,35,69.473571,2,3,4,26,4,2018,2018-4-26,"20,2","0,0"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222739,60111371,601113713,1,2,0.708580,46.428103,1,38,46.428103,1,7,1,4,6,2018,2018-6-4,"16,2","0,0"
222740,60111371,601113714,1,2,3.237211,46.428103,1,38,46.428103,1,7,1,4,6,2018,2018-6-4,"16,2","0,0"
222741,60111371,601113715,4,5,2.769001,46.428103,1,38,46.428103,1,7,1,4,6,2018,2018-6-4,"16,2","0,0"
222742,60111371,601113716,4,2,3.893271,46.428103,1,38,46.428103,1,7,1,4,6,2018,2018-6-4,"16,2","0,0"


In [45]:
df_transp['prec'].value_counts()

prec
0,0     119719
Ip        9315
1,0       8274
0,3       7259
0,1       7218
0,2       6851
7,4       5129
28,9      4730
2,1       4229
13,5      4118
22,9      4044
2,7       3869
19,9      3825
5,6       3700
17,6      3650
2,6       3584
19,2      3583
4,2       3574
5,3       3522
17,3      3366
2,4       2360
13,8      2316
2,8       1998
3,5       1697
1,5        808
0,8          6
Name: count, dtype: int64

In [48]:
### Convert tmed and prec to float
df_transp['tmed']=df_transp['tmed'].str.replace(",",".").astype(float)
df_transp['prec']=df_transp['prec'].str.replace("Ip","0") ### We count rain of < 0.1 mm as dry weather.
df_transp['prec']=df_transp['prec'].str.replace(",",".").astype(float)
df_transp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 222744 entries, 0 to 222743
Data columns (total 18 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   id_indiv    222744 non-null  object 
 1   id_trip     222744 non-null  object 
 2   freq        222744 non-null  int64  
 3   reason      222744 non-null  int64  
 4   distance    222744 non-null  float64
 5   trip_pond   222744 non-null  float64
 6   gender      222744 non-null  int64  
 7   age         222744 non-null  int64  
 8   indiv_pond  222744 non-null  float64
 9   spanish     222744 non-null  int64  
 10  studies     222744 non-null  int64  
 11  activity    222744 non-null  int64  
 12  day         222744 non-null  int64  
 13  month       222744 non-null  int64  
 14  year        222744 non-null  int64  
 15  fechamerge  222744 non-null  object 
 16  tmed        222744 non-null  float64
 17  prec        222744 non-null  float64
dtypes: float64(5), int64(10), object(3)
memory u

In [8]:
df_transp = pd.read_csv("./data/treated/transp.csv")
df_transp.groupby("id_indiv")["id_trip"].count()
df_transp[df_transp["id_indiv"]==60111371]

,id_indiv,id_trip,start_trip,freq,reason,distance,trip_pond,gender,age,spanish,studies,activity,day,month,year,datemerge,tmed,prec
222737,60111371,601113711,900,2,3,4.181772,46.428103,1,38,1,7,1,4,6,2018,2018-6-4,"16,2","0,0"
222738,60111371,601113712,1030,2,3,2.633570,46.428103,1,38,1,7,1,4,6,2018,2018-6-4,"16,2","0,0"
222739,60111371,601113713,1243,1,2,0.708580,46.428103,1,38,1,7,1,4,6,2018,2018-6-4,"16,2","0,0"
222740,60111371,601113714,1530,1,2,3.237211,46.428103,1,38,1,7,1,4,6,2018,2018-6-4,"16,2","0,0"
222741,60111371,601113715,1700,4,5,2.769001,46.428103,1,38,1,7,1,4,6,2018,2018-6-4,"16,2","0,0"
222742,60111371,601113716,1930,4,2,3.893271,46.428103,1,38,1,7,1,4,6,2018,2018-6-4,"16,2","0,0"
222743,60111371,601113717,2300,1,2,3.237211,46.428103,1,38,1,7,1,4,6,2018,2018-6-4,"16,2","0,0"
